In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pyspark.sql.functions as f



In [2]:
spark = SparkSession.builder.master("local[*]").appName("cas2").getOrCreate()
spark

## Weather Data

In [3]:
sc = spark.sparkContext 

#!!! MAKE SURE THAT YOUR TEXTFILE HEADER IS THIS EXACT STRING: !!!
"STATION_ID,DATE,LOCATION,WIND,TMP,DEW,SLP"

weather_path = 'out_2020_header_changed.txt'


In [4]:
df1 = spark.read.option("header", "true").csv(weather_path)

# Apply transformations and filter abnormal wind data columns
df_wind_data_counts = df1.withColumn("splitcount", f.size(f.split(df1.WIND, " ")))
# Filter out columns which do not have 5 elements in the wind information
df_filtered_wind_data_counts = df_wind_data_counts.filter("splitcount == 5")
# wind_split =df_filtered_wind_data_counts.select(f.split(df_filtered_wind_data_counts.WIND," ")).rdd.flatMap(lambda x: x).toDF(schema=["wind_angle", "wind_qualityAngle", "wind_type", "wind_speed", "wind_qualitySpeed"])
## Wind_split
dfnew = df_filtered_wind_data_counts.withColumn('wind_angle', f.split(df_filtered_wind_data_counts['WIND'], ' ').getItem(0)).withColumn('wind_qualityAngle', f.split(df_filtered_wind_data_counts['WIND'], ' ').getItem(1)).withColumn('wind_type', f.split(df_filtered_wind_data_counts['WIND'], ' ').getItem(2)).withColumn('wind_speed', f.split(df_filtered_wind_data_counts['WIND'], ' ').getItem(3)).withColumn('wind_qualitySpeed', f.split(df_filtered_wind_data_counts['WIND'], ' ').getItem(4))
dfnew.show()

# wind_split.show()

# dfnew = df_filtered_wind_data_counts.join(wind_split)
# dfnew.show()

+-----------+----------+-----------------+--------------+-------+-------+-------+----------+----------+-----------------+---------+----------+-----------------+
| STATION_ID|      DATE|         LOCATION|          WIND|    TMP|    DEW|    SLP|splitcount|wind_angle|wind_qualityAngle|wind_type|wind_speed|wind_qualitySpeed|
+-----------+----------+-----------------+--------------+-------+-------+-------+----------+----------+-----------------+---------+----------+-----------------+
|99773799999|2020-04-20|SILVER BAY  MN US|260 1 N 0088 1|+0081 1|+9999 9|09973 1|         5|       260|                1|        N|      0088|                1|
|99773799999|2020-04-21|SILVER BAY  MN US|330 1 N 0082 1|+0028 1|+9999 9|10020 1|         5|       330|                1|        N|      0082|                1|
|99773799999|2020-04-22|SILVER BAY  MN US|170 1 N 0026 1|+0017 1|+9999 9|10146 1|         5|       170|                1|        N|      0026|                1|
|99773799999|2020-04-23|SILVER BAY

In [5]:
#Split other columns into multiples
dfnewer = dfnew.withColumn('tmp_val', f.split(dfnew['TMP'], ' ').getItem(0)).withColumn('tmp_quality', f.split(dfnew['TMP'], ' ').getItem(1))
dfnewer = dfnewer.withColumn('dew_val', f.split(dfnewer['DEW'], ' ').getItem(0)).withColumn('dew_quality', f.split(dfnewer['DEW'], ' ').getItem(1))
dfnewer = dfnewer.withColumn('slp_val', f.split(dfnewer['SLP'], ' ').getItem(0)).withColumn('slp_quality', f.split(dfnewer['SLP'], ' ').getItem(1))
dfnewer.show()

+-----------+----------+-----------------+--------------+-------+-------+-------+----------+----------+-----------------+---------+----------+-----------------+-------+-----------+-------+-----------+-------+-----------+
| STATION_ID|      DATE|         LOCATION|          WIND|    TMP|    DEW|    SLP|splitcount|wind_angle|wind_qualityAngle|wind_type|wind_speed|wind_qualitySpeed|tmp_val|tmp_quality|dew_val|dew_quality|slp_val|slp_quality|
+-----------+----------+-----------------+--------------+-------+-------+-------+----------+----------+-----------------+---------+----------+-----------------+-------+-----------+-------+-----------+-------+-----------+
|99773799999|2020-04-20|SILVER BAY  MN US|260 1 N 0088 1|+0081 1|+9999 9|09973 1|         5|       260|                1|        N|      0088|                1|  +0081|          1|  +9999|          9|  09973|          1|
|99773799999|2020-04-21|SILVER BAY  MN US|330 1 N 0082 1|+0028 1|+9999 9|10020 1|         5|       330|             

In [6]:
df_good_cols = dfnewer.drop("splitcount", "WIND", "TMP", "DEW", "SLP")
df_good_cols.show()

+-----------+----------+-----------------+----------+-----------------+---------+----------+-----------------+-------+-----------+-------+-----------+-------+-----------+
| STATION_ID|      DATE|         LOCATION|wind_angle|wind_qualityAngle|wind_type|wind_speed|wind_qualitySpeed|tmp_val|tmp_quality|dew_val|dew_quality|slp_val|slp_quality|
+-----------+----------+-----------------+----------+-----------------+---------+----------+-----------------+-------+-----------+-------+-----------+-------+-----------+
|99773799999|2020-04-20|SILVER BAY  MN US|       260|                1|        N|      0088|                1|  +0081|          1|  +9999|          9|  09973|          1|
|99773799999|2020-04-21|SILVER BAY  MN US|       330|                1|        N|      0082|                1|  +0028|          1|  +9999|          9|  10020|          1|
|99773799999|2020-04-22|SILVER BAY  MN US|       170|                1|        N|      0026|                1|  +0017|          1|  +9999|       

In [7]:
df_date_split = df_good_cols.withColumn('year', f.split(df_good_cols['DATE'], '-').getItem(0)).withColumn('month', f.split(df_good_cols['DATE'], '-').getItem(1)).withColumn('day', f.split(df_good_cols['DATE'], '-').getItem(2))
df_date_split = df_date_split.drop("DATE")
df_date_split.show()

+-----------+-----------------+----------+-----------------+---------+----------+-----------------+-------+-----------+-------+-----------+-------+-----------+----+-----+---+
| STATION_ID|         LOCATION|wind_angle|wind_qualityAngle|wind_type|wind_speed|wind_qualitySpeed|tmp_val|tmp_quality|dew_val|dew_quality|slp_val|slp_quality|year|month|day|
+-----------+-----------------+----------+-----------------+---------+----------+-----------------+-------+-----------+-------+-----------+-------+-----------+----+-----+---+
|99773799999|SILVER BAY  MN US|       260|                1|        N|      0088|                1|  +0081|          1|  +9999|          9|  09973|          1|2020|   04| 20|
|99773799999|SILVER BAY  MN US|       330|                1|        N|      0082|                1|  +0028|          1|  +9999|          9|  10020|          1|2020|   04| 21|
|99773799999|SILVER BAY  MN US|       170|                1|        N|      0026|                1|  +0017|          1|  +999

In [14]:
from pyspark.sql.functions import col, avg
df_date_split.select(avg('month')).collect()

[Row(avg(month)=6.492774998657439)]

## Stocks data

In [93]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType,TimestampType
stocks_file = "Stocks.txt"
schema = StructType([StructField('_c0', StringType(), False),\
                     StructField('_c1', StringType(), False),\
                     StructField('_c2', FloatType(), False),\
                     StructField('_c3', IntegerType(), False),])

df_stocks = spark.read.option("header", "false").schema(schema).csv(stocks_file)




df_stocks =  df_stocks.selectExpr(
    '_c0 AS Stock',
    '_c1 AS Date',
    '_c2 AS Price',
    '_c3 AS Volume',
)


# df_stocks = df_stocks.withColumn("Price",df_stocks.Price.cast('float'))
# df_stocks = df_stocks.withColumn("Volume",df_stocks.Volume.cast('int'))

df_stocks.show()
df_stocks.printSchema()

+--------------------+----------+-----+------+
|               Stock|      Date|Price|Volume|
+--------------------+----------+-----+------+
|32843.Nordamerika...|01/01/2016|18.98| 50635|
|32843.Nordamerika...|01/04/2016|18.52| 51616|
|32843.Nordamerika...|01/05/2016|19.15| 54898|
|32843.Nordamerika...|01/06/2016|19.71| 41555|
|32843.Nordamerika...|01/07/2016|19.17| 44430|
|32843.Nordamerika...|01/08/2016|18.94| 72673|
|32843.Nordamerika...|01/11/2016| 19.1| 45426|
|32843.Nordamerika...|01/12/2016|19.39| 61457|
|32843.Nordamerika...|01/13/2016|19.27| 61805|
|32843.Nordamerika...|01/14/2016|19.17| 35597|
|32843.Nordamerika...|01/15/2016|18.81| 69227|
|32843.Nordamerika...|01/18/2016|18.81| 69227|
|32843.Nordamerika...|01/19/2016|18.82| 23700|
|32843.Nordamerika...|01/20/2016|17.97| 41439|
|32843.Nordamerika...|01/21/2016|17.82| 35240|
|32843.Nordamerika...|01/22/2016|18.62| 79139|
|32843.Nordamerika...|01/25/2016|18.09| 89251|
|32843.Nordamerika...|01/26/2016|17.62| 75300|
|32843.Nordam

In [94]:
df_stocks_dates = df_stocks.withColumn('month', f.split(df_stocks['Date'], '/').getItem(0).cast('int')).withColumn('day', f.split(df_stocks['Date'], '/').getItem(1).cast('int')).withColumn('year', f.split(df_stocks['Date'], '/').getItem(2).cast('int'))
# df_stocks_dates = df_stocks_dates.drop("Date")
df_stocks_dates.show()


+--------------------+----------+-----+------+-----+---+----+
|               Stock|      Date|Price|Volume|month|day|year|
+--------------------+----------+-----+------+-----+---+----+
|32843.Nordamerika...|01/01/2016|18.98| 50635|    1|  1|2016|
|32843.Nordamerika...|01/04/2016|18.52| 51616|    1|  4|2016|
|32843.Nordamerika...|01/05/2016|19.15| 54898|    1|  5|2016|
|32843.Nordamerika...|01/06/2016|19.71| 41555|    1|  6|2016|
|32843.Nordamerika...|01/07/2016|19.17| 44430|    1|  7|2016|
|32843.Nordamerika...|01/08/2016|18.94| 72673|    1|  8|2016|
|32843.Nordamerika...|01/11/2016| 19.1| 45426|    1| 11|2016|
|32843.Nordamerika...|01/12/2016|19.39| 61457|    1| 12|2016|
|32843.Nordamerika...|01/13/2016|19.27| 61805|    1| 13|2016|
|32843.Nordamerika...|01/14/2016|19.17| 35597|    1| 14|2016|
|32843.Nordamerika...|01/15/2016|18.81| 69227|    1| 15|2016|
|32843.Nordamerika...|01/18/2016|18.81| 69227|    1| 18|2016|
|32843.Nordamerika...|01/19/2016|18.82| 23700|    1| 19|2016|
|32843.N

In [95]:
df_stocks_dates.printSchema()

root
 |-- Stock: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Price: float (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- year: integer (nullable = true)



## SparkSQL Test

In [97]:
stock1 = "32843.Nordamerika_USA-NASDAQ_CRA-International-Inc._CRAI"
stock2 = "41574.Nordamerika_USA-OTC_Polydex-Pharmaceuticals_POLXF"
stock3 = "23349.Europa_Schweden_BlackPearl-Resources-Inc.-Reg.-Shares-SDRs-1-o._02155Z"
df_stock_1 = df_stocks_dates.filter(f"Stock = '{stock1}'").limit(10).registerTempTable("stock1")
df_stock_2 = df_stocks_dates.filter(f"Stock = '{stock2}'").limit(10).registerTempTable("stock2")
df_stock_3 = df_stocks_dates.filter(f"Stock = '{stock3}'").limit(10).registerTempTable("stock3")

# spark.sql("CACHE TABLE testCache1 OPTIONS ('storageLevel' 'DISK_ONLY') SELECT * FROM stock1")
# spark.sql("CACHE TABLE testCache2 OPTIONS ('storageLevel' 'DISK_ONLY') SELECT * FROM stock2")
# spark.sql("CACHE TABLE testCache3 OPTIONS ('storageLevel' 'DISK_ONLY') SELECT * FROM stock3")

spark.sql("CACHE TABLE stock1")
spark.sql("CACHE TABLE stock2")
spark.sql("CACHE TABLE stock3")


# UNCOMMENT IF FIRST TIME
spark.sql(\
  "CREATE TEMP VIEW COLS3 AS SELECT \
  stock1.volume X, stock2.volume Y1, stock3.volume Y2 FROM stock1\
  INNER JOIN stock2 ON stock1.day = stock2.day\
   INNER JOIN stock3 ON stock1.Date = stock3.Date"\
     ).show()




++
||
++
++



In [98]:
spark.sql("SELECT * FROM COLS3").show()

spark.sql("SELECT *, CASE WHEN Y1 >= Y2 THEN 'ONE' WHEN Y2 > Y1 THEN 'TWO' END AS Y_star FROM COLS3").show()

+-----+-----+------+
|    X|   Y1|    Y2|
+-----+-----+------+
|50635|12000|200784|
|51616|  400|170002|
|54898|  100| 66159|
|41555| 3200| 66159|
|44430| 4976|387422|
|72673| 4976|121141|
|45426|  100| 90265|
|61457|  100|128003|
|61805|  100| 51492|
+-----+-----+------+

+-----+-----+------+------+
|    X|   Y1|    Y2|Y_star|
+-----+-----+------+------+
|50635|12000|200784|   TWO|
|51616|  400|170002|   TWO|
|54898|  100| 66159|   TWO|
|41555| 3200| 66159|   TWO|
|44430| 4976|387422|   TWO|
|72673| 4976|121141|   TWO|
|45426|  100| 90265|   TWO|
|61457|  100|128003|   TWO|
|61805|  100| 51492|   TWO|
+-----+-----+------+------+

